## Modeling the storm surge using Dean and Dalrymple method 1992

In [3]:
# import necessary libraries
from numpy import *
import scipy
from scipy import optimize 
from pylab import *
from ipywidgets import Button, Text, Label, Box, Layout
from IPython.display import clear_output
import matplotlib.pyplot as plt
depthText = Text(placeholder='Input Offshore Water Depth ')
windVelText = Text(placeholder='Input Wind Velocity ')
bedSloText = Text(placeholder='Input Bed Slope')
plotBtn = Button(description='Show plot and save result', button_style = 'success', layout= Layout(
    display = 'flex',
    flex_flow = 'row',
    justify_content = 'center',
    width = '180px',
    disabled=False
))

input_layout = Layout(
    flex_flow = 'column',
    align_items = 'stretch',
    disabled = False
)
label_layout = Layout(
    width = '160px'
)

text_layout = Layout( 
    display = 'flex',
    flex_flow = 'row',
    justify_content = 'flex-start'
 #   width = '200%'
)

def plot_stormheight(x, h, eta, h0, u10, m):
    # Import everything from matplotlib (numpy is accessible via 'np' alias)

    # Create a new figure of size 8x6 points, using 80 dots per inch
    figure(figsize=(20, 12), dpi=80)

    # Create a new subplot from a grid of 1x1
    fig, ax = plt.subplots(1)

    # Plot cosine using blue color with a continuous line of width 1 (pixels)
    ax.fill_between(-(x - x[-1]) / 1000, eta, 0, color="red", alpha=0.5, lw=2, label='Water Depth')
    ax.fill_between(-(x - x[-1]) / 1000, 0, -h, color="blue", alpha=0.5, lw=2, label='Storm Surge')
    ax.fill_between(-(x - x[-1]) / 1000, -h, -h[0], color="yellow", alpha=0.3, lw=2, label='Bed Slope')

    p_surge = Rectangle((0, 0), 1, 1, fc="red", alpha=0.5)
    p_water = Rectangle((0, 0), 1, 1, fc="blue", alpha=0.5)
    p_slope = Rectangle((0, 0), 1, 1, fc="yellow", alpha=0.3)

    legend([p_surge, p_water, p_slope], ["Storm Surge", "Still Water", "Seabed"], shadow=True, fancybox=True,
        numpoints=5, loc=3)
    
    figtext(0.45,0.3,
        "Water Depth  : %.2f m\n"
        "Wind Velocity: %.2f m/s\n"
        "Bed Slope    : %f\n"
        "Storm Surge  : %.2f m" % (h0, u10, m, eta[-1]),
        horizontalalignment='left', verticalalignment='center', fontsize='large',
        family='monospace',
        color='black', weight='normal', bbox=dict(boxstyle="round,pad=0.5", ec='black', fill=False)
    )
    ax.set_xlabel('Distance from Shoreline (km)')
    ax.set_ylabel('Elevation above Mean Sea Level (m)')
    ax.grid()
    
    # Save figure using 90 dots per inch
    savefig("stormsurge.png", dpi=80)
    # Show result on screen
    plt.show()
    



def plotBtn_clicked(a):
    #% h0=40; % deep water depth, It was 42 m for Katrina
    #% u10=40; % wind velocity 10 meter above the ground, It was 40 m/s for Katrina
    #% m=0.00084; % bed slope = h0/l;
    h0 = float(depthText.value)
    u10 = float(windVelText.value)
    m = float(bedSloText.value)
    n = 1.3
    Row = 1000
    Roa = 1.204
    if bedSloText.value == 0:
        l = 50000
    else:
        l = h0 / m
        
    x = linspace(0, l, 1000)
    if u10 >= 40:
        k = (0.8 + 0.065 * 40) / 1000
    else:
        k = (0.80 + 0.065 * u10) / 1000   
    Tauwind = Roa * k * u10 ** 2
    A = n * Tauwind * l / (Row * 9.81 * h0 ** 2)
    eta = array([0.0 for i in range(len(x))])
    
    if m == 0:
    # calculate eta for bed without slope
        eta = h0 * (sqrt(1 + 2 * A * x / l) - 1)
    else:
    # calculate eta for bed with slope
        h = -m * x + h0
        eta[0] = 0.0
    
   
    
    fileX = open('eta.txt','w')

    for i in range(1, len(x)):
    # initial value for eta from previous step
        def f(eta1):
            return x[i] / l - ((1.0 - (h[i] + eta1) / h0) - A * log(((h[i] + eta1) / h0 - A) / (1 - A)))
        etaini = eta[i - 1]
        eta[i] = scipy.optimize.fsolve(f, etaini)[0]
        fileX.write(str(eta[i])+'\n')
    fileX.close()
    
    plot_stormheight(x, h, eta, h0, u10, m)
    
plotBtn.on_click(plotBtn_clicked)


inputBox = Box(
    [Box([Label(value='Offshore Water Depth (m) : ',layout = label_layout), depthText], layout = text_layout),
    Box([Label(value='Wind Velocity (m/s) : ',layout = label_layout), windVelText], layout = text_layout),
    Box([Label(value='Bed Slope : ',layout = label_layout), bedSloText], layout = text_layout), 
    Box([plotBtn], layout= Layout(
    display = 'flex',
    flex_flow = 'row',
    justify_content = 'flex-end',
    width = '468px',
    disabled=False))], 
    layout = input_layout)

inputBox

Box(children=(Box(children=(Label(value='Offshore Water Depth (m) : ', layout=Layout(width='160px')), Text(val…